<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from IPython.display import IFrame

IFrame(src='https://voodoohop.github.io/colabasaservice/', width=700, height=600)

In [ ]:
!pip install deep-daze stomp.py imageio-ffmpeg --upgrade

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 

flatten = itertools.chain.from_iterable

TEXT="A black and white square"

DEFAULT_PARAMS='--lower_bound_cutout 0.4 --image-width=512 --iterations=500 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True'

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)



def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)

    headers = { 
        "params": params,
        "text": TEXT
    }
    conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          headers = {
              **headers,         
              "type": mimetype,
              "filename": filename
              }
          conn.send(body=base64_encoded_media, headers=headers, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))
run(TEXT)
conn.disconnect()



Deleting ./A_black_and_white_square.000266.jpg before starting.
Deleting ./A_black_and_white_square.000490.jpg before starting.
Deleting ./A_black_and_white_square.000453.jpg before starting.
Deleting ./A_black_and_white_square.000150.jpg before starting.
Deleting ./A_black_and_white_square.000080.jpg before starting.
Deleting ./A_black_and_white_square.000314.jpg before starting.
Deleting ./A_black_and_white_square.000274.jpg before starting.
Deleting ./A_black_and_white_square.000300.jpg before starting.
Deleting ./A_black_and_white_square.000354.jpg before starting.
Deleting ./A_black_and_white_square.000066.jpg before starting.
Deleting ./A_black_and_white_square.000438.jpg before starting.
Deleting ./A_black_and_white_square.000175.jpg before starting.
Deleting ./A_black_and_white_square.000240.jpg before starting.
Deleting ./A_black_and_white_square.000369.jpg before starting.
Deleting ./A_black_and_white_square.000477.jpg before starting.
Deleting ./A_black_and_white_square.0001

# New section

In [ ]:
%%javascript
// Create the channel object
const channel = new BroadcastChannel('my-channel');

// Set up a listener
channel.onmessage = message => console.log('Received:', message.data);

// Post a message to all subscribers
channel.postMessage('Hello, world');

In [ ]:
conn.disconnect()